In [ ]:
import requests 
import json 
import pandas as pd

# we need to define a dictionary with the series IDs by city / series:
categories = ['housing', 'food', 'apparel', 'transportation', 'medical care', 'recreation', 'education and communication']
category_codes = ['AH', 'AF11', 'AA', 'AT', 'AM', 'AR', 'AE']
cities = ['Atlanta', 'Baltimore', 'Boston', 'Chicago', 'Dallas', 'Denver', 'Detroit', 'Houston', 
          'Los Angeles', 'Miami', 'Minneapolis', 'New York', 'Philadelphia', 'Phoenix', 'Riverside', 
          'St. Louis', 'San Diego', 'San Francisco', 'Seattle', 
          'Tampa', 'Urban Alaska', 'Urban Hawaii', 'Washington']
city_codes = ['S35CS', 'S35ES', 'S11AS', 'S23AS', 'S37AS', 'S48BS', 'S23BS', 'S37BS',
              'S49AS', 'S35BS', 'S24AS', 'S12AS', 'S12BS', 'S48AS', 'S49CS',
              'S24BS', 'S49ES', 'S49BS', 'S49DS', 
              'S35DS', 'S49GS', 'S49FS', 'A311S']
'''series_ids['Atlanta'] = ['housing': 'CUURS35CSAH',
                        'food': 'CUURS35CSAF1',
                        'apparel': 'CUURS35CSAA',
                        'transportation': 'CUURS35CSAT',
                        'medical care': 'CUURS35CSAM',
                        'recreation': 'CUURS35CSAR',
                        'education and communication': 'CUURS35CSAE'' '''
BLS_API_URL = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'
api_key = 'f45aaf3b4d9c4d069132a033657b7234'
data_frames = []
for category, cat_code in zip(categories, category_codes):
    for city, city_code in zip(cities, city_codes):
        series_id = f'CUUR{city_code}{cat_code}'
        # Prepare the API request
        headers = {'Content-type': 'application/json'}
        request_data = {
            'seriesid': [series_id],
            'registrationkey': api_key,
            'startyear': '2019',
            'endyear': '2024'
        }
        # Fetch data from the API
        response = requests.post(BLS_API_URL, json=request_data, headers=headers)
        # Check for a valid response
        if response.status_code == 200:
            # Parse JSON response
            data = response.json()
            # Extract relevant data
            series_data = data['Results']['series'][0]['data']
            df = pd.DataFrame(series_data)
            # Add category and city for context
            df['category'] = category
            df['city'] = city
            df['series_id'] = series_id
            # Keep only relevant columns and rename
            if df.shape[0] > 0:
                df = df[['category', 'city', 'series_id', 'year', 'period', 'value']]
                df.rename(columns={'value': 'price_index'}, inplace=True)
                # Append to the list of DataFrames
                data_frames.append(df)
            else:
                print(f"No data available for {series_id}")
        else:
            print(f"Failed to fetch data for {series_id}: {response.status_code}")

# Combine all DataFrames into a single panel DataFrame
panel_df = pd.concat(data_frames, ignore_index=True)

# Output the final DataFrame
print(panel_df)

# Save to CSV for further analysis
panel_df.to_csv("bls_panel_datav1.csv", index=False)

# now group by category and city to find the cumulative inflation (in percentage) from january 2021 to november 2024


No data available for CUURA311SAH
No data available for CUURA311SAF11
No data available for CUURA311SAA
No data available for CUURA311SAT
No data available for CUURA311SAM
No data available for CUURA311SAR
No data available for CUURA311SAE
                         category          city    series_id  year period  \
0                         housing       Atlanta  CUURS35CSAH  2024    M10   
1                         housing       Atlanta  CUURS35CSAH  2024    M08   
2                         housing       Atlanta  CUURS35CSAH  2024    M06   
3                         housing       Atlanta  CUURS35CSAH  2024    M04   
4                         housing       Atlanta  CUURS35CSAH  2024    M02   
...                           ...           ...          ...   ...    ...   
6698  education and communication  Urban Hawaii  CUURS49FSAE  2019    M09   
6699  education and communication  Urban Hawaii  CUURS49FSAE  2019    M07   
6700  education and communication  Urban Hawaii  CUURS49FSAE  2019 

OSError: [Errno 30] Read-only file system: 'bls_panel_data.csv'

In [2]:
panel_df.to_csv("bls_panel_datav1.csv", index=False)

OSError: [Errno 30] Read-only file system: 'bls_panel_datav1.csv'